##### Define Libraries and Values

In [1]:
import ipywidgets as widgets

In [21]:
Text1 = widgets.Output()
with Text1:
    print("Choose your File, that you want to edit:")
    
Text2 = widgets.Output()
with Text2:
    print("\nChoose the focus of the EQ bandpass:")
    
Text3 = widgets.Output()
with Text3:
    print("\nChoose the EQ Slider Count:")
    
Text4 = widgets.Output()
with Text4:
    print("\nEqualizer:")

In [3]:
Bands = 10
GraphicEQ = widgets.GridspecLayout(Bands, 1)
Controllrange = [-12, 12, 0]

#### Widgets

In [4]:
# Upload a file
Upload = widgets.FileUpload(
    accept='',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False,  # True to accept multiple files upload else False
    tooltips=['Upload a .wav File to Edit it']
)

In [5]:
# Choose EQ Preset
SetFilterType = widgets.ToggleButtons(
    options=['Bass', 'Mid', 'High'],
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Focus on 200Hz', 'Focus on 1kHz', 'Focus on 4kHz']
)

In [6]:
# Choose Band Count
ChooseBandCount = widgets.BoundedIntText(
    value=Bands,
    min=5,
    max=20,
    step=5,
    disabled=False
)

In [7]:
Freqspecturm = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
#if SetFilterType == 'Bass':
    #Freqspecturm = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
#if SetFilterType == 'Mid':
    #Freqspecturm = [21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40]

In [23]:
# Builds EQ 
def Build_Equalizer (SetFilterType, ChooseBandCount):
    # print((SetFilterType))
    # print((Bands))
    
    GraphicEQ = widgets.GridspecLayout(Bands, 1)
    
    #Calculates Freqspectrum with SetFilterType and ChooseBandCount
    
    #Temp
    if SetFilterType == 'Bass':
        Freqspecturm = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
    if SetFilterType == 'Mid':
        Freqspecturm = [21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40]
    if SetFilterType == 'High':
        Freqspecturm = [41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60]
        
    
    # Calculate Frequency steps
    slider = [widgets.IntSlider(
        value=Controllrange[2],
        min=Controllrange[0],
        max=Controllrange[1],
        step=1,
        description=str(Freqspecturm[i])+' Hz',
        disabled=False,
        continuous_update=False,
        orientation='vertical',
        readout=True,
        readout_format='d',
        layout = widgets.Layout(width='50%')
    ) for i in range(int(ChooseBandCount))]
    
    EQDisplay = widgets.Box(children=slider)
    
    display(EQDisplay)

## Display Programm Interface



In [24]:
# Combine and Format all Widgets
UserInterface = widgets.VBox(children = [Text1, Upload, Text2, SetFilterType, Text3, ChooseBandCount, Text4, GraphicEQ])
EQValues = widgets.interactive_output(Build_Equalizer, {'SetFilterType': SetFilterType, 'ChooseBandCount': ChooseBandCount})
#FilterType = widgets.interactive_output(Calculate_Frequencies, {'SetFilterType': SetFilterType})

display(UserInterface, EQValues, FilterType)

Output()

Output()

### Sources

#### Example Audio File Credits
    Song: Dimension 
    Creator: Creo 
    Youtube: https://www.youtube.com/channel/UCsCWA3Y3JppL6feQiMRgm6Q 
    Website: https://creo-music.com/track/dimension
    Licensed under: https://creativecommons.org/licenses/by/4.0/

#### Programming
    https://ipywidgets.readthedocs.io/en/latest/examples/Using%20Interact.html
    https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html
    https://en.wikibooks.org/wiki/LaTeX/Mathematics

#### Formulars and Documents for Audio
    https://www.teachmeaudio.com/mixing/techniques/audio-spectrum#upper-midrange
    
#### Other Programms used
    Music editing: Audacity
    Music Download: https://github.com/yt-dlp/yt-dlp
    
#### Special Thanks
    https://www.senarclens.eu/~gerald/

<br>Upload file
<br>Split Audio into its Passes
<br>Redo if EQ Mode is changed
<br>
<br>Display EQ Sliders
<br>Combine Audio Parts back together
<br>Display Output Plots
<br>Dispay finished file

### TestArea

In [11]:
a = widgets.IntSlider()
b = widgets.IntSlider()
c = widgets.IntSlider()
ui = widgets.HBox([a, b, c])
def f(a, b, c):
    print((a, b, c))

out = widgets.interactive_output(f, {'a': a, 'b': b, 'c': c})

display(ui, out)

Output()

In [12]:
x_widget = widgets.FloatSlider(min=0.0, max=10.0, step=0.05)
y_widget = widgets.FloatSlider(min=0.5, max=10.0, step=0.05, value=5.0)

def update_x_range(*args):
    x_widget.max = 2.0 * y_widget.value
y_widget.observe(update_x_range, 'value')

def printer(x, y):
    print(x, y)
widgets.interact(printer,x=x_widget, y=y_widget);

interactive(children=(FloatSlider(value=0.0, description='x', max=10.0, step=0.05), FloatSlider(value=5.0, des…